In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
a = []
while(1):
    a.append('1')

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

AUTOTUNE = tf.data.experimental.AUTOTUNE
print("AUTOTUNE: ", AUTOTUNE)

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

TensorFlow 2.x selected.
Num GPUs Available:  1
Found GPU at: /device:GPU:0
AUTOTUNE:  -1


In [3]:
import pathlib
project_dir = "/content/drive/My Drive/master_uppsala/p3_computer-assisted_image_analysis_2_project/"
oral_cancer_dataset_dir = project_dir + "OralCancer_DataSet3/"
train_dir = pathlib.Path(oral_cancer_dataset_dir + "train/")

CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])
print(CLASS_NAMES)
total_train = 73303  # len(list(train_dir.glob('*/*/*.jpg')))  # 73303
total_test = 55514  # len(list(test_dir.glob('*/*/*.jpg')))  # 55514

BATCH_SIZE = 128
epochs = 30
IMG_HEIGHT = 80
IMG_WIDTH = 80

# from IPython.display import display
# image_example_dir = str(list(train_dir.glob('Cancer/000/*.jpg'))[0])
# display(Image.open(image_example_dir))

['Cancer' 'Healthy']


In [0]:
train_record_file = oral_cancer_dataset_dir + 'train_images.tfrecords'
test_record_file = oral_cancer_dataset_dir + 'test_images.tfrecords'

train_image_dataset = tf.data.TFRecordDataset(train_record_file)
test_image_dataset = tf.data.TFRecordDataset(test_record_file)
# Create a dictionary describing the features.
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

train_ds = train_image_dataset.map(_parse_image_function)
test_ds = test_image_dataset.map(_parse_image_function)

# for a in test_ds.take(1):
#   raw_image = a['image_raw'].numpy()
#   label = a['label'].numpy()
#   print(label)
#   display.display(display.Image(raw_image))

In [0]:
def decode_img_label(x):
  def _decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)  # convert the compressed string to a 3D uint8 tensor
    return tf.image.convert_image_dtype(img, tf.float32)  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  
  img = _decode_img(x['image_raw'])
  label = (CLASS_NAMES == CLASS_NAMES[0]).astype(int) == x['label']
  return img, label

train_labeled_ds = train_ds.map(decode_img_label, num_parallel_calls=AUTOTUNE)
test_labeled_ds = test_ds.map(decode_img_label, num_parallel_calls=AUTOTUNE)

# for b in test_labeled_ds.take(1):
#   print(b[0].numpy())
#   print(b[1].numpy())

In [0]:
# train_labeled_ds = train_labeled_ds.cache()
# test_labeled_ds = test_labeled_ds.cache()f

def augment(ds):
  lr_flip_ds = ds.map(lambda x,y: (tf.image.flip_left_right(x),y), num_parallel_calls=AUTOTUNE)
  ud_flip_ds = ds.map(lambda x,y: (tf.image.flip_up_down(x),y), num_parallel_calls=AUTOTUNE)
  rot90_ds = ds.map(lambda x,y: (tf.image.rot90(x),y), num_parallel_calls=AUTOTUNE)
  rot180_ds = ds.map(lambda x,y: (tf.image.rot90(x, k=2),y), num_parallel_calls=AUTOTUNE)
  rot270_ds = ds.map(lambda x,y: (tf.image.rot90(x, k=3),y), num_parallel_calls=AUTOTUNE)
  lr_flip_rot90_ds = lr_flip_ds.map(lambda x,y: (tf.image.rot90(x),y), num_parallel_calls=AUTOTUNE)
  lr_flip_rot270_ds = lr_flip_ds.map(lambda x,y: (tf.image.rot90(x, k=3),y), num_parallel_calls=AUTOTUNE)
  return ds.concatenate(lr_flip_ds).concatenate(ud_flip_ds).concatenate(rot90_ds).concatenate(rot180_ds).concatenate(rot270_ds).concatenate(lr_flip_rot90_ds).concatenate(lr_flip_rot270_ds)

train_augmented_ds = augment(train_labeled_ds)
test_augmented_ds = augment(test_labeled_ds)

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  # ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  # ds = ds.repeat()  # Repeat forever
  ds = ds.batch(1)
  ds = ds.prefetch(buffer_size=AUTOTUNE)  # `prefetch` lets the dataset fetch batches in the background while the model is training.
  return ds

train_model_ds = prepare_for_training(train_augmented_ds, cache=False)
test_model_ds = prepare_for_training(test_augmented_ds, cache=False)

In [0]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

image_batch, label_batch = next(iter(train_model_ds))
show_batch(image_batch.numpy(), label_batch.numpy())

In [0]:
dn_model = DenseNet121(
    include_top=False, weights='imagenet', input_tensor=None, 
    input_shape=None, pooling=None)

# dimensions of our images.
img_width, img_height = 80, 80

top_model_weights_path = project_dir + 'densenet_features/bottleneck_fc_model.h5'

# the predict_generator method returns the output of a model, given a generator that yields batches of numpy data
# bottleneck_features_train = dn_model.predict(train_model_ds, steps=total_train*8, verbose=1)
# densenet_train_feature_file = project_dir + 'densenet_features/bottleneck_features_train.npy'
# np.save(open(densenet_train_feature_file, 'wb'), bottleneck_features_train)

bottleneck_features_validation = dn_model.predict(test_model_ds, steps=total_test*8, verbose=1)
# densenet_validation_feature_file = project_dir + 'densenet_features/bottleneck_features_validation.npy'
# np.save(open(densenet_validation_feature_file, 'wb'), bottleneck_features_validation)

 22792/444112 [>.............................] - ETA: 2:09:25

In [4]:
densenet_train_feature_file = project_dir + 'densenet_features/bottleneck_features_train.npy'
# densenet_validation_feature_file = project_dir + 'densenet_features/bottleneck_features_validation.npy'

nb_train_canser_samples, nb_train_healthy_samples = 22453, 50850
nb_validation_canser_samples, nb_validation_healthy_samples = 20322, 35192

train_data = np.load(open(densenet_train_feature_file, 'rb'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array(([0] * nb_train_canser_samples + [1] * nb_train_healthy_samples)*8)

# validation_data = np.load(open(densenet_validation_feature_file, 'rb'))
validation_data = bottleneck_features_validation
validation_labels = np.array(([0] * nb_validation_canser_samples + [1] * nb_validation_healthy_samples)*8)

FileNotFoundError: ignored

In [0]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=BATCH_SIZE,
          validation_data=(validation_data, validation_labels))
model.save_weights(project_dir + '/densenet_features/bottleneck_fc_model.h5')